# Greenhouse Gas Emissions

## Introduction

This dataset was taken from https://ourworldindata.org/co2-and-greenhouse-gas-emissions and details the annual emission of various greenhouse gases (GHGs) from state and non-state entities since the year 1850. It focuses on the three main GHGs:

- CO₂ (carbon dioxide / Kohlendioxid)
- CH₄ (methane / Methan)
- N₂O (nitrous oxide / Distickstoffmonoxid)

The emission amounts are always expressed in tonnes (1 ton = 1000 kg). In order to meaningfully compare the emissions from different gases, the CH4 and N20 amounts are expressed in tonnes of CO2 equivalents.

The goal of the current analysis is to investigate how GHG emissions are distributed over time, across countries, and by type of gas. This will consist three main steps:

- Data Collection and Cleaning
- Feature Engineering
- Data analysis and Visualization

In [1]:
# import necessary packages

import numpy as np
import pandas as pd

## Data Cleaning and Exploration 

In the following, we will get better acquainted with our dataset and perform some data cleaning where necessary.

In [ ]:
# load dataset and remove duplicates -> there are no duplicates

df = pd.read_csv(r"./../data/ghg-emissions-by-gas.csv")
print(df.size)
df.drop_duplicates() # no effect on dataset
print(df.size)

249120
249120


In [3]:
df.head(5)

,Entity,Code,Year,Annual nitrous oxide emissions in CO₂ equivalents,Annual methane emissions in CO₂ equivalents,Annual CO₂ emissions
0,Afghanistan,AFG,1850,223008.40,3594926.5,3520884.0
1,Afghanistan,AFG,1851,227659.61,3615134.5,3561188.2
2,Afghanistan,AFG,1852,232190.92,3635346.8,3596619.0
3,Afghanistan,AFG,1853,236528.19,3655563.5,3630340.0
4,Afghanistan,AFG,1854,240597.22,3675785.0,3662827.5


The column names contain subscript characters which are difficult to type, so I'm going to change those names:

In [4]:
# rename columns
old_column_names = list(df);
new_column_names = ['entity', 'code', 'year', 'n2o_in_co2_equivalents_annual', 'ch4_in_co2_equivalents_annual', 'co2_annual'] 

new_columns_dict = dict(zip(old_column_names, new_column_names))
print(new_columns_dict)

{'Entity': 'entity', 'Code': 'code', 'Year': 'year', 'Annual nitrous oxide emissions in CO₂ equivalents': 'n2o_in_co2_equivalents_annual', 'Annual methane emissions in CO₂ equivalents': 'ch4_in_co2_equivalents_annual', 'Annual CO₂ emissions': 'co2_annual'}


In [5]:
# rename columns

df = df.rename(columns=new_columns_dict)

df.head()

,entity,code,year,n2o_in_co2_equivalents_annual,ch4_in_co2_equivalents_annual,co2_annual
0,Afghanistan,AFG,1850,223008.40,3594926.5,3520884.0
1,Afghanistan,AFG,1851,227659.61,3615134.5,3561188.2
2,Afghanistan,AFG,1852,232190.92,3635346.8,3596619.0
3,Afghanistan,AFG,1853,236528.19,3655563.5,3630340.0
4,Afghanistan,AFG,1854,240597.22,3675785.0,3662827.5




Here's an overview of the dataset. We can see that there are null values in the **country code**, **N2O**, and **CH4** columns. We will look at the more precisely later.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41520 entries, 0 to 41519
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   entity                         41520 non-null  object 
 1   code                           37541 non-null  object 
 2   year                           41520 non-null  int64  
 3   n2o_in_co2_equivalents_annual  38060 non-null  float64
 4   ch4_in_co2_equivalents_annual  37195 non-null  float64
 5   co2_annual                     41520 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.9+ MB


Which period of time does our dataset cover?

In [7]:
print(df['year'].unique().min(), df['year'].unique().max())

1850 2022


It would be also interesting to know the total GHG emission for each country and year. To this end, we define a new column in which we sum the amounts of CO2, CH4, and N2O. For the latter two we use of course tonnes of CO2 equivalents.

In [8]:
df['total_ghg_in_co2_equivalents_annual'] = df[new_column_names[3]] + df[new_column_names[4]] + df[new_column_names[5]]
df.head()

,entity,code,year,n2o_in_co2_equivalents_annual,ch4_in_co2_equivalents_annual,co2_annual,total_ghg_in_co2_equivalents_annual
0,Afghanistan,AFG,1850,223008.40,3594926.5,3520884.0,7338818.90
1,Afghanistan,AFG,1851,227659.61,3615134.5,3561188.2,7403982.31
2,Afghanistan,AFG,1852,232190.92,3635346.8,3596619.0,7464156.72
3,Afghanistan,AFG,1853,236528.19,3655563.5,3630340.0,7522431.69
4,Afghanistan,AFG,1854,240597.22,3675785.0,3662827.5,7579209.72


Now we look closer at the `NaN`'s corresponding to the country code column. They correspond mostly to various country aggregates, such as continents or the European Union:

In [9]:
# display non-country entities

(df[df['code'].isna()].groupby(['entity'])).agg({'total_ghg_in_co2_equivalents_annual': ['min','max','sum']})

total_ghg_in_co2_equivalents_annual  \
                                                                         min   
entity                                                                         
Africa                                                          2.058945e+08   
Asia                                                            1.261978e+09   
Asia (excl. China and India)                                    5.385385e+08   
Europe                                                          1.312901e+09   
Europe (excl. EU-27)                                            7.593475e+08   
Europe (excl. EU-28)                                            5.659444e+08   
European Union (27)                                             5.535537e+08   
European Union (28)                                             7.468454e+08   
High-income countries                                           1.860330e+09   
Kuwaiti Oil Fires                                                        NaN   
Least developed countries (Jones et al.)                        2.812132e+08   
Leeward Islands                                                          NaN   
Low-income countries                                            1.411458e+08   
Lower-middle-income countries                                   7.547854e+08   
North America                                                   1.025436e+09   
North America (excl. USA)                                       1.450048e+08   
OECD (Jones et al.)                                             1.902137e+09   
Oceania                                                         7.731356e+07   
Panama Canal Zone                                                        NaN   
Ryukyu Islands                                                           NaN   
South America                                                   1.995812e+08   
St. Kitts-Nevis-Anguilla                                                 NaN   
Upper-middle-income countries                                   1.375916e+09   

                                                                      
                                                   max           sum  
entity                                                                
Africa                                    4.927793e+09  2.774420e+11  
Asia                                      3.006286e+10  1.188670e+12  
Asia (excl. China and India)              1.220878e+10  6.006800e+11  
Europe                                    1.027650e+10  8.369748e+11  
Europe (excl. EU-27)                      5.448046e+09  4.261745e+11  
Europe (excl. EU-28)                      4.645106e+09  3.246777e+11  
European Union (27)                       5.200339e+09  4.108004e+11  
European Union (28)                       6.075526e+09  5.122971e+11  
High-income countries                     1.745190e+10  1.470511e+12  
Kuwaiti Oil Fires                                  NaN  0.000000e+00  
Least developed countries (Jones et al.)  3.316723e+09  1.930884e+11  
Leeward Islands                                    NaN  0.000000e+00  
Low-income countries                      2.072128e+09  1.151289e+11  
Lower-middle-income countries             1.021281e+10  5.506154e+11  
North America                             9.061696e+09  7.834488e+11  
North America (excl. USA)                 2.085793e+09  1.549667e+11  
OECD (Jones et al.)                       1.709812e+10  1.477415e+12  
Oceania                                   9.353508e+08  6.676216e+10  
Panama Canal Zone                                  NaN  0.000000e+00  
Ryukyu Islands                                     NaN  0.000000e+00  
South America                             5.015533e+09  2.967848e+11  
St. Kitts-Nevis-Anguilla                           NaN  0.000000e+00  
Upper-middle-income countries             2.627348e+10  1.313801e+12

The are 5 entities in this list are not aggregates:

- Kuwaiti Oil Fires
- Leeward Islands
- Panama Canal Zone
- Ryukyu Islands
- St. Kitts-Nevis-Anguilla

They also seem to have some `NaN`'s, so let's investigate in more detail where the null values occur.

In [10]:
small_entity_list = ['Kuwaiti Oil Fires', 'Leeward Islands', 'Panama Canal Zone', 'Ryukyu Islands', 'St. Kitts-Nevis-Anguilla']

print(df.loc[df['entity'].isin(small_entity_list), ('n2o_in_co2_equivalents_annual', 'ch4_in_co2_equivalents_annual', 'co2_annual')].isna().sum())

n2o_in_co2_equivalents_annual    865
ch4_in_co2_equivalents_annual    865
co2_annual                         0
dtype: int64


The null values correspond to the N2O and CH4. How does the CO2 column for these 5 entities look then?

In [11]:
df.loc[df['entity'].isin(small_entity_list), ('entity', 'co2_annual')]

,entity,co2_annual
19722,Kuwaiti Oil Fires,0.0
19723,Kuwaiti Oil Fires,0.0
19724,Kuwaiti Oil Fires,0.0
19725,Kuwaiti Oil Fires,0.0
19726,Kuwaiti Oil Fires,0.0
...,...,...
35806,St. Kitts-Nevis-Anguilla,0.0
35807,St. Kitts-Nevis-Anguilla,0.0
35808,St. Kitts-Nevis-Anguilla,0.0
35809,St. Kitts-Nevis-Anguilla,0.0


It seems like there are many zeroes for the CO2 emissions. Are there actually any non-zero values? It turns out that there are some, but not so many:

In [12]:
# check how many years have co2 > 0 for the small entities

print('Number of years with non-null co2 emissions for entity...\n')
for i in range(len(small_entity_list)):
    print(f'{small_entity_list[i]}:', df.loc[(df['entity'] == small_entity_list[i]) & (df['co2_annual'] > 0), 'co2_annual'].count())

Number of years with non-null co2 emissions for entity...

Kuwaiti Oil Fires: 1
Leeward Islands: 7
Panama Canal Zone: 30
Ryukyu Islands: 21
St. Kitts-Nevis-Anguilla: 24


Are these yearly contributions to CO2 emissions signficant? To answer this question, let's find the largest among them:

In [13]:
# check amount of co2 emissions for the small entities

df_small_entities = df.loc[(df['entity'].isin(small_entity_list)) & (df['co2_annual'] > 0), ('entity', 'year', 'co2_annual')]
df_small_entities = df_small_entities.sort_values('co2_annual', ascending = False)
df_small_entities

,entity,year,co2_annual
19863,Kuwaiti Oil Fires,1991,477924830.0
31435,Ryukyu Islands,1972,9005072.0
31434,Ryukyu Islands,1971,3226945.0
31433,Ryukyu Islands,1970,1904242.0
29537,Panama Canal Zone,1977,1132176.0
...,...,...,...
35747,St. Kitts-Nevis-Anguilla,1959,14656.0
35750,St. Kitts-Nevis-Anguilla,1962,14656.0
35746,St. Kitts-Nevis-Anguilla,1958,10992.0
35748,St. Kitts-Nevis-Anguilla,1960,10992.0


Let's compare the largest yearly CO2 emission from the 'small entities' dataset from above to the correpsonding world CO2 yearly emission. The result indicated that only approx. 2% of the global CO2 emissions in 1991 came from the Kuwaiti Oil Fires. 

In [14]:
world_co2_1991 = df.loc[(df['entity'] == 'World') & (df['year'] == 1991), 'co2_annual'].sum()
kof_co2_1991 = df.loc[(df['entity'] == 'Kuwaiti Oil Fires') & (df['year'] == 1991), 'co2_annual'].sum()

'{:.2%}'.format(kof_co2_1991/world_co2_1991)

'1.73%'

### Selecting the countries from the dataset

We are interested mainly in the countries from the dataset, so let's select and save them in a new dataset.

In [15]:
df_countries = df[df['code'].notna()] # this contains also the aggregate 'world'
df_countries.head(5)

,entity,code,year,n2o_in_co2_equivalents_annual,ch4_in_co2_equivalents_annual,co2_annual,total_ghg_in_co2_equivalents_annual
0,Afghanistan,AFG,1850,223008.40,3594926.5,3520884.0,7338818.90
1,Afghanistan,AFG,1851,227659.61,3615134.5,3561188.2,7403982.31
2,Afghanistan,AFG,1852,232190.92,3635346.8,3596619.0,7464156.72
3,Afghanistan,AFG,1853,236528.19,3655563.5,3630340.0,7522431.69
4,Afghanistan,AFG,1854,240597.22,3675785.0,3662827.5,7579209.72


Let's look again at null values. Once more, the N2O and CH4 columns look problematic.

In [16]:
df_countries.isna().sum()

entity                                    0
code                                      0
year                                      0
n2o_in_co2_equivalents_annual          2595
ch4_in_co2_equivalents_annual          3460
co2_annual                                0
total_ghg_in_co2_equivalents_annual    3460
dtype: int64

To which countries correspond the null value entries?

In [17]:
mask_na = (df_countries['ch4_in_co2_equivalents_annual'].isna()) | (df_countries['n2o_in_co2_equivalents_annual'].isna())
df_countries.loc[mask_na, 'entity'].unique()

array(['Anguilla', 'Antarctica', 'Aruba', 'Bermuda',
       'Bonaire Sint Eustatius and Saba', 'British Virgin Islands',
       'Christmas Island', 'Curacao', 'Faroe Islands', 'French Polynesia',
       'Greenland', 'Kosovo', 'Marshall Islands', 'Montserrat',
       'New Caledonia', 'Palestine', 'Saint Helena',
       'Saint Pierre and Miquelon', 'Sint Maarten (Dutch part)',
       'Wallis and Futuna'], dtype=object)

The first impulse would be to remove these countries from the dataset. However we should first check if we are missing any important GHG contributions if we do so. To this end, we calculate the total GHG emission between 1850-2022 for each of these countries and compare it to the global GHG emission for the same time period. In this process, we replace any null values with zero.

In [18]:
# global GHG emission 1850-2022
ghg_total_world = df_countries.loc[df_countries['entity'] == 'World', 'total_ghg_in_co2_equivalents_annual'].sum()

# set the null values to zero
df_na_countries = df_countries.loc[mask_na, ['entity', 'n2o_in_co2_equivalents_annual', 'ch4_in_co2_equivalents_annual', 'co2_annual']].fillna(0)
# calculate the new column coresponging to total GHG emissions
df_na_countries['ghg_total'] = df_na_countries['n2o_in_co2_equivalents_annual'] + df_na_countries['ch4_in_co2_equivalents_annual'] + df_na_countries['co2_annual']

# perform a sum over the time period 1850-2022
df_na_countries = df_na_countries.loc[:,['entity','ghg_total']].groupby('entity').sum('ghg_total')

# express the GHG emission for each country as a percentage of the global GHG emission
df_na_countries['ghg_total_world_percentage'] = (df_na_countries['ghg_total'] / ghg_total_world).map('{:.4%}'.format)

df_na_countries.sort_values('ghg_total', ascending = False)

,ghg_total,ghg_total_world_percentage
entity,,
Curacao,5.849795e+08,0.0170%
New Caledonia,1.613994e+08,0.0047%
Kosovo,1.218219e+08,0.0035%
Aruba,7.812265e+07,0.0023%
Sint Maarten (Dutch part),6.692907e+07,0.0019%
Greenland,3.424397e+07,0.0010%
French Polynesia,3.221889e+07,0.0009%
Faroe Islands,3.220022e+07,0.0009%
Bermuda,2.981037e+07,0.0009%


We see that the countries with null value entries have an insignificant contribution to the global GHG emissions. Therefore we can safely drop them from the dataset:

In [19]:
# final step of cleaning => can safely drop nan's

df_countries_clean = df_countries.dropna()
df_countries_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34081 entries, 0 to 41519
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   entity                               34081 non-null  object 
 1   code                                 34081 non-null  object 
 2   year                                 34081 non-null  int64  
 3   n2o_in_co2_equivalents_annual        34081 non-null  float64
 4   ch4_in_co2_equivalents_annual        34081 non-null  float64
 5   co2_annual                           34081 non-null  float64
 6   total_ghg_in_co2_equivalents_annual  34081 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 2.1+ MB


For data analysis purposes, let's create a new dataset in which we save the cumulative GHG emission by year for each country.

In [20]:
# rename columns
col_list = ['entity', 'n2o_in_co2_equivalents_annual', 'ch4_in_co2_equivalents_annual', 'co2_annual', 'total_ghg_in_co2_equivalents_annual']
col_rename_list = ['n2o_in_co2_equivalents_cumul', 'ch4_in_co2_equivalents_cumul', 'co2_cumul', 'ghg_in_co2_equivalents_cumul']
col_rename_dict = dict(zip(col_list[1:],col_rename_list))

df_countries_cumul = df_countries_clean
df_countries_cumul = df_countries_cumul.rename(columns = col_rename_dict)


# perform the cumulative sum for each country
for country in list(df_countries_clean['entity'].unique()):
    df_countries_cumul.loc[df_countries_cumul['entity'] == country, col_rename_list] = df_countries_cumul.loc[df_countries_cumul['entity'] == country, col_rename_list].cumsum()

df_countries_cumul

,entity,code,year,n2o_in_co2_equivalents_cumul,ch4_in_co2_equivalents_cumul,co2_cumul,ghg_in_co2_equivalents_cumul
0,Afghanistan,AFG,1850,2.230084e+05,3.594926e+06,3.520884e+06,7.338819e+06
1,Afghanistan,AFG,1851,4.506680e+05,7.210061e+06,7.082072e+06,1.474280e+07
2,Afghanistan,AFG,1852,6.828589e+05,1.084541e+07,1.067869e+07,2.220696e+07
3,Afghanistan,AFG,1853,9.193871e+05,1.450097e+07,1.430903e+07,2.972939e+07
4,Afghanistan,AFG,1854,1.159984e+06,1.817676e+07,1.797186e+07,3.730860e+07
...,...,...,...,...,...,...,...
41515,Zimbabwe,ZWE,2018,3.646332e+08,8.189308e+08,2.230478e+09,3.414042e+09
41516,Zimbabwe,ZWE,2019,3.697828e+08,8.317511e+08,2.248859e+09,3.450393e+09
41517,Zimbabwe,ZWE,2020,3.744702e+08,8.442249e+08,2.264779e+09,3.483474e+09
41518,Zimbabwe,ZWE,2021,3.794366e+08,8.570978e+08,2.280829e+09,3.517364e+09


We export the datasets for later stages of the analysis:

In [21]:
# df_countries_clean.to_csv('ghg_per_year_clean.csv', index=False)
# df_countries_cumul.to_csv('ghg_cumul_clean.csv', index=False)

Finally, let us perform a consistency check of our dataset to make sure that dropping null value entries was a sound decision. We therefore compare the cumulated GHG emission value for the pre-existing aggregate 'World' with one that we calculate from the rest of the dataset by summing over all countries.

In [22]:
mask_1 = (df_countries_cumul['entity'] == 'World') & (df_countries_cumul['year'] == 2022)
mask_2 = (df_countries_cumul['entity'] != 'World') & (df_countries_cumul['year'] == 2022)

series_1 = df_countries_cumul.loc[mask_1, col_rename_list]
series_2 = df_countries_cumul.loc[mask_2, col_rename_list].sum()

(1 - series_2/series_1).map('{:.4%}'.format)

,n2o_in_co2_equivalents_cumul,ch4_in_co2_equivalents_cumul,co2_cumul,ghg_in_co2_equivalents_cumul
41000,0.0002%,0.0000%,0.0480%,0.0351%


The sub-permille errors are more than tolerable, so we can rest assured that our dataset is consistent.